In [1]:
## test riffusion img 2 img pipeline
import io
from pathlib import Path

import PIL

from datatypes import Img2ImgInput, InferenceOutput
from riffusion_img2img_pipeline import RiffusionImg2ImgPipeline
from spectrogram_image_converter import SpectrogramImageConverter
from spectrogram_params import SpectrogramParams
from util import base64_util

: 

: 

In [ ]:
# Load the inial image by ID
seed_images_dir = "/Users/zachary/Desktop/Stanford/_Win2023/EE\ 269/Project/mel-generator/mel-generator/riffusion-reference/seed_images"
init_image_path = Path(seed_images_dir, f"vibes.png")
init_image = PIL.Image.open(str(init_image_path)).convert("RGB")

# setup model input
inputs = Img2ImgInput(
         text_prompt= "generate a saxaphone melody in C major",
         seed=0,
         init_spectrogram=init_image,
         mask_image=None,
         negative_prompt=None,
         denoising=0.75,
         guidance=7.0,
         ddim_steps=50,
         ddim_eta=0.0,
    )

# load pipeline 
pipeline = RiffusionImg2ImgPipeline.load_checkpoint(
    checkpoint="riffusion/riffusion-model-v1",
    use_traced_unet=True,
    device="cuda",
)

# Execute the model to get the spectrogram image
image = pipeline.riffuse(
    inputs,
    use_reweighting=True,
)

In [ ]:
# Reconstruct audio from the image
params = SpectrogramParams(
    min_frequency=0,
    max_frequency=10000,
)
converter = SpectrogramImageConverter(params=params, device=str(pipeline.device))
segment = converter.audio_from_spectrogram_image(
    image,
    apply_filters=True,
)

# Export audio to MP3 bytes
mp3_bytes = io.BytesIO()
segment.export(mp3_bytes, format="mp3")
mp3_bytes.seek(0)

# Export image to JPEG bytes
image_bytes = io.BytesIO()
image.save(image_bytes, exif=image.getexif(), format="JPEG")
image_bytes.seek(0)

# Assemble the output dataclass
output = InferenceOutput(
    image="data:image/jpeg;base64," + base64_util.encode(image_bytes),
    audio="data:audio/mpeg;base64," + base64_util.encode(mp3_bytes),
    duration_s=segment.duration_seconds,
)